First Goal is to make the computer understand the language<br>

Here are the steps

1. Sentence segmentation
2. Word tokenization
3. Stemming
4. Lemmatization
5. Stop word analysis
6. Dependency parsing
7. Part-of-speech (POS) tagging

In [148]:
import numpy as np
import re

**Prepare a proper storage**

In [149]:
class Tweet:
    def __init__(self, count, hate_speech_count, offensive_language_count, neither_count, classs, tweet):
        """
        - count (int): The total count of the tweet.
        - hate (int): The count of hate speech in the tweet.
        - offensive (int): The count of offensive language in the tweet.
        - neither (int): The count of content classified as neither hate speech nor offensive.
        - classs (str): The classification of the tweet.
        - tweet (str): The text content of the tweet.
        """
        self.count = count
        self.hate = hate_speech_count
        self.offensive = offensive_language_count
        self.neither = neither_count
        self.classs = classs
        self.tweet = tweet

    def __str__(self):
        return f"{self.count} ; {self.hate} ; {self.offensive} ; {self.neither} ; {self.classs} ;; {self.tweet}"

**Load the data**

In [150]:
Tweets = []

In [151]:
with open('./archive/train.csv', 'r+') as file:
    previous_line = ''

    # Initialize a list to accumulate the modified content
    final_content_lines = []

    # Read and accumulate non-empty lines
    for line in file:
        stripped_line = line.strip()

        if stripped_line and stripped_line[0].isdigit():
            # If the current line is not empty and starts with an integer, accumulate it
            final_content_lines.append(stripped_line)
            previous_line = stripped_line
        else:
            # If the current line doesn't start with an integer, append it to the previous line
            previous_line += stripped_line

In [152]:
for line in final_content_lines:
    
    comma_indices = [index for index, char in enumerate(line) if char == ',']

    # Extracting substrings between commas
    count_str = line[0:comma_indices[0]].strip()
    hate_str = line[comma_indices[0]+1:comma_indices[1]].strip()
    offensive_str = line[comma_indices[1]+1:comma_indices[2]].strip()
    neither_str = line[comma_indices[2]+1:comma_indices[3]].strip()
    classs_str = line[comma_indices[3]+1:comma_indices[4]].strip()
    tweet_str = line[comma_indices[4]+1:].strip()

    # Converting to integers
    count = int(count_str) if count_str.isdigit() else None
    hate = int(hate_str) if hate_str.isdigit() else None
    offensive = int(offensive_str) if offensive_str.isdigit() else None
    neither = int(neither_str) if neither_str.isdigit() else None
    classs = int(classs_str) if classs_str.isdigit() else None

    # Creating an instance of the Tweet class
    tweet_instance = Tweet(count, hate, offensive, neither, classs_str, tweet_str)

    # Append the tweet instance to a list or do whatever you need to do with it
    Tweets.append(tweet_instance)

In [153]:
for t in Tweets[:10]:
    print(t)

3 ; 0 ; 0 ; 3 ; 2 ;; !!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
3 ; 0 ; 3 ; 0 ; 1 ;; !!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
3 ; 0 ; 3 ; 0 ; 1 ;; !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3 ; 0 ; 2 ; 1 ; 1 ;; !!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
6 ; 0 ; 6 ; 0 ; 1 ;; !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
3 ; 1 ; 2 ; 0 ; 1 ;; "!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just blows me..claim you so faithful and down for somebody but still fucking with hoes! &#128514;&#128514;&#128514;"""
3 ; 0 ; 3 ; 0 ; 1 ;; "!!!!!!""@__BrighterDays: I can not just sit up and HATE on another bitch .. I got too much shit going on!"""
3 ; 0 ; 3 ; 0 ; 1 ;; !!!!&#8220

**Tweet formatting**

remove all points when it's not the end of a sentence

In [154]:
def remove_points(line):
    l = list(line)

    for i in range(len(l)):
        if i < len(l) - 2 and l[i] == '.' and 'A' <= l[i+2] <= 'Z':
            l[i] = ' '

    line = ''.join(l)

apply first format

In [155]:
for t in Tweets:
    remove_points(t.tweet)
    t.tweet = t.tweet   .replace('RT', '').replace('!', '').replace('"', '').replace("\n", '')\
                        .replace(';', ' ').replace('-', ' ').replace(' and ', ' & ').replace('\'', '')\
                        .replace('?', '.').replace(',', '')
    for _ in range(4):
        t.tweet = t.tweet.replace('  ', ' ').replace('..', '.').replace(' .', '.') # remove multiple points & space
    
    if t.tweet and t.tweet[0] == ' ':
        t.tweet = t.tweet[1:]


for t in Tweets[:10]:
    print(t)


3 ; 0 ; 0 ; 3 ; 2 ;; @mayasolovely: As a woman you shouldnt complain about cleaning up your house. &amp as a man you should always take the trash out.
3 ; 0 ; 3 ; 0 ; 1 ;; @mleew17: boy dats cold.tyga dwn bad for cuffin dat hoe in the 1st place
3 ; 0 ; 3 ; 0 ; 1 ;; @UrKindOfBrand Dawg @80sbaby4life: You ever fuck a bitch & she start to cry. You be confused as shit
3 ; 0 ; 2 ; 1 ; 1 ;; @C_G_Anderson: @viva_based she look like a tranny
6 ; 0 ; 6 ; 0 ; 1 ;; @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361 
3 ; 1 ; 2 ; 0 ; 1 ;; @T_Madison_x: The shit just blows me.claim you so faithful & down for somebody but still fucking with hoes &#128514 &#128514 &#128514 
3 ; 0 ; 3 ; 0 ; 1 ;; @__BrighterDays: I can not just sit up & HATE on another bitch. I got too much shit going on
3 ; 0 ; 3 ; 0 ; 1 ;; &#8220 @selfiequeenbri: cause Im tired of you big bitches coming for us skinny girls&#8221 
3 ; 0 ; 3 ; 0 ; 1 ;; &amp you might n

Let's change:
- all `@<username>` to `username`
- all URLs to `weblink`
- all `&amp` to `&`

In [156]:
for t in Tweets:
    t.tweet = re.sub(r'@([a-zA-Z0-9_]+)', 'username', t.tweet)
    t.tweet = re.sub(r'https?://\S+', 'weblink', t.tweet)
    t.tweet = re.sub(r'&amp', '&', t.tweet)

    if t.tweet and t.tweet[0] == '.':
        t.tweet = t.tweet[1:]

Change everything in **lowercase**

In [157]:
# for t in Tweets:
#     t.tweet = t.tweet.lower()

In [158]:
for t in Tweets[:1000]:
    print(t.tweet)

username: As a woman you shouldnt complain about cleaning up your house. & as a man you should always take the trash out.
username: boy dats cold.tyga dwn bad for cuffin dat hoe in the 1st place
username Dawg username: You ever fuck a bitch & she start to cry. You be confused as shit
username: username she look like a tranny
username: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361 
username: The shit just blows me.claim you so faithful & down for somebody but still fucking with hoes &#128514 &#128514 &#128514 
username: I can not just sit up & HATE on another bitch. I got too much shit going on
&#8220 username: cause Im tired of you big bitches coming for us skinny girls&#8221 
& you might not get ya bitch back & thats that 
username :hobbies include: fighting Mariam
Keeks is a bitch she curves everyone lol I walked into a conversation like this. Smh
Murda Gang bitch its Gang Land 
So hoes that smoke are losers. yea. go on IG
bad 